# Computational Chemistry

This demonstration aims to introduce how Python can be applied in a Chemistry context. We will focus on reactions and kinetics, by utilising a dataset and analysing it to visualise trends.

The data used in this demo has been taken from this GitHub repository: https://github.com/flboudoire/chemical-kinetics/blob/master/examples/simple_example/data/concentrations%20vs%20time.csv, with documentation here: https://chemical-kinetics.readthedocs.io/en/latest/index.html (file used is `concentrations_vs_time.csv`)

---

## How to use Jupyter

In Jupyter notebooks code can be run cell by cell. Cells are like textboxes which let you input code and comments, and the play button (⏵) lets you run cells individually. The fast forward button (⏩) lets you run the entire notebook. You can also press Shift+Enter to run cells individually.

Some of the cells can be run as they are and others may require you to input some code before running. It will be specified which you need to do.

---

## Concentration and time data

In this demo we will have a look at a different set of data for a typical chemical reaction that might take place for a kinetics study, where the progress of the reaction is monitored over time. A common way to do this is look at the loss in mass of the reactant over time (if it is producing a gaseous product) and use the known starting mass to calculate the concentrations. We will then determine the rate order of the reactant (zeroth, first, second), by plotting graphs from integrated rate laws. We will then use _machine learning_ to predict the concentration of one of the reagents at a given time.

Machine learning relates to using computers to extract information from data, automatically learning and improving through the use of that data. It is a broad term that applies to a wide range of methods, and one of the simplest techniques is the linear fit, or _linear regression_. This uses the data given to create a line of best fit for the graph, and we can then use this to extrapolate or predict data points outside the current given range, as well as interpolate (predicting data points within the range).

First, we will import the _libraries_ that we require - these are pre-written packages that have various different functionalities. Most of them are imported in the format `import X as Y`, where `X` is the full name of the library and `Y` is an abbreviation for it - you can put anything you want as the name for `Y`, but there is normally a naming convention for each library, such as `pandas` being shortened to `pd`, to create consistency across code.

In [ ]:
# Run this cell as it is
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import rdkit
from sklearn.linear_model import LinearRegression

In [ ]:
# Run this cell as it is
data_filename_2 = "data/concentrations_vs_time.csv" # Create variable to store the filename
data_2 = pd.read_csv(data_filename_2, sep = ",") # Read in the data using pandas, specifying that the values are separated by a comma

We have created a variable called `data` which contains a table of information that we have extracted from a file. We can preview the first few rows of the table using `.head()`.

In [ ]:
# Run this cell as it is
data_2.head()

The dataset contains columns for time (`t`) in seconds, and the concentrations in mol dm$^{-3}$ for 3 different compounds (arbitrarily named `A`, `B` and `C`). We can create variables for each of these columns and then plot them on a graph to look at the behaviour these compounds over time.

In [ ]:
# Run this cell as it is
# Create variables for each column in the dataframe
t = data_2["t"]
A = data_2["A"]
B = data_2["B"]
C = data_2["C"]

# Plot a scatter graph for time vs concentration
plt.scatter(t, A, label = "A")
plt.scatter(t, B, label = "B")
plt.scatter(t, C, label = "C")
plt.xlabel("time / s")
plt.ylabel("concentration / mol dm$^{-3}$")
plt.legend() # Adds a legend (key) to the plot to identify which points belong to which label
plt.show() # Ensures both the plot and legend are showing

Here from the graph, we can see that A is likely to be the reactant as it shows a decrease over time. C is likely to be the product as it shows an increase over time. B seems to show a mix of both, suggesting that it is an intermediate - formed when A reacts, and then reacting itself to form C. 

The plot mainly shows clean trends, but we can see that for C the points become bunched together towards the end of the graph. We can use something called a rolling average to clean up the points.

In [ ]:
# Run this cell as it is
data_rolling_average = data_2.rolling(5, min_periods = 1).mean() # try adjusting this number
t_rolling_average = data_rolling_average["t"]
A_rolling_average = data_rolling_average["A"]
B_rolling_average = data_rolling_average["B"]
C_rolling_average = data_rolling_average["C"]
plt.scatter(t_rolling_average, A_rolling_average, label = "A")
plt.scatter(t_rolling_average, B_rolling_average, label = "B")
plt.scatter(t_rolling_average, C_rolling_average, label = "C")
plt.xlabel("time / s")
plt.ylabel("concentration / mol dm$^{-3}$")
plt.legend()
plt.show()

The parameter (value inside the brackets for `data_2.rolling()`) is the window that the rolling average is taken for. Say we are calculating the rolling average over a window of 5 rows of data - at the very beginning, there aren't yet 5 values to use and there will only be a point on the graph after 5 data points are in the window. As a solution, we can also set the `min_periods` - as the rolling average is only taken after the window number is reached there will be points missing to start off with so the `min_periods` parameter takes this into account. Something to note is that the larger the number, the smoother the plot becomes, but if the number is too large (such as 20), the graph begins to lose its shape.

**Question (1): Experiment with the parameters in the `.rolling()` function in the cell above, and then re-run it. Have a look at how the shape of the graph might be different.**

As you may know from A-level Chemistry, reactants in a chemical reaction can be zeroth, first or second order depending on how concentration varies with time. You should be familiar with the first set of equations, the differentiated rate law where you can calculate the rate from the concentrations and rate constant, as well as the units for the rate constant (bottom row). The second set, the integrated rate law, may be new to you - but essentially this just integrates the first set of equations so that you can calculate the concentration of reactant at a given time, given you have the initial concentrations. The graphs for each order and type of rate law also each have distinctive shapes.

![reaction rates](reaction_rates.png)

We will be referring to this throughout the demo.

Focusing on the blue plot, since we have established that reactant A is the reactant, its graph shape is likely to either be first order or second order since it shows curvature. However, in order to determine which one it is exactly, we will need to make use of integrated rate laws.
These equations are in a disguised $y = mx + c$ format and each rate order has a unique integrated rate equation that will give a linear plot.

Looking at the integrated rate equations, we can see that for a first order reaction, we plot $ln([A])$ as our $y$ vs $t$ as our $x$. The gradient is $-k$ (the negative of the rate constant). The $y$ intercept will be $[A]_0$. For a second order reaction, we plot $\frac{1}{[A]}$ as our $y$ vs $t$ as our $x$. The gradient is $+k$ (rate constant). The $y$ intercept will also be $[A]_0$. We will not go into the details of the integrations in this demo, but you can find out more about how these equations are derived here: https://chemistrytalk.org/integrated-rate-laws/

We can plot both of these graphs, adding an extra column for these calculations, and see which graph gives a linear plot. We can't start variable names with numbers or have them contain operators like /, so we've named the variable for `1/[A]` to be `recA` (as short for 'reciprocal of A')

We'll first create these new columns which save the results of calculations done to existing columns. We will preview our dataset again, and can see the 2 new columns:

In [ ]:
# Run this cell as it is
data_2["ln([A])"] = np.log(data_2["A"]) # np.log() is essentially the ln() function in maths
data_2["1/[A]"] = 1 / (data_2["A"])
data_2.head()

We will now plot the graphs:

In [ ]:
# Run this cell as it is
lnA = data_2["ln([A])"]
recA = data_2["1/[A]"]

fig, ax = plt.subplots()
ax.scatter(t,lnA)
ax.set_xlabel('t / s')
ax.set_ylabel('ln([A] / mol dm$^{-3}$)')

fig, ax = plt.subplots()
ax.scatter(t,recA)
ax.set_xlabel('t / s')
ax.set_ylabel('1 / ([A] / mol dm$^{-3}$)')

We can see that this compares quite well to the pictures of the graph shapes. The second order graph appears to give a more linear plot. We can also verify this quantitatively using the linear regression tool `scikit-learn` by calculating an $R^2$ value. The $R^2$ value ranges from 0 to 1 and measures how well the model predicts the outcome (0 = very poor, 1 = very well). The more linear the graph, the better the model predicts the outcome.

In [ ]:
# Run this cell as it is
model_1 = LinearRegression(fit_intercept=True)
x = data_2[["t"]] # The double square brackets are needed because input features must be 2D
y = data_2['ln([A])']
model_1.fit(x, y)
R_squared_1 = model_1.score(x, y)
print(f"R squared is: {R_squared_1}")

In [ ]:
# Run this cell as it is
model_2 = LinearRegression(fit_intercept=True)
x = data_2[["t"]]
y = data_2['1/[A]']
model_2.fit(x, y)
R_squared_2 = model_2.score(x, y)
print(f"R squared is: {R_squared_2}")

We can see that both graphs give pretty good $R^2$ values, however as model 2's value is closer to 1 so we can conclude that the reaction is second order with respect to reactant A.

We can now use `scikit learn` again, but this time we use linear regression to predict the concentration of reactant at a given time. We can use it to plot a line of best fit from the graph and using this, find the gradient and y intercept from the model, which are the $m$ and $c$ values respectively of a $y = mx + c$ equation.

In [ ]:
# Run this cell as it is
from sklearn import linear_model

# Prepare the data
X = data_2[["t"]]
Y = data_2["1/[A]"]

model = linear_model.LinearRegression(fit_intercept = True) # Create the model

model.fit(X, Y) # Fit the model to the data

data_2["pred_y"] = model.predict(X) # Use the model for prediction and create new column in the dataframe to store this

# Output the results
print("Line gradient from model: ", model.coef_[0])
print("Line intercept from model:", model.intercept_)

The `pred_y` column contains the points for this line of best fit and so we can plot this on the graph (shown in black).

In [ ]:
# Run this cell as it is
lnA = data_2["ln([A])"]
recA = data_2["1/[A]"]
pred_Y = data_2["pred_y"]
fig, ax = plt.subplots()
ax.scatter(t,recA)
ax.plot(t,pred_Y,color = "black")
ax.set_xlabel('t / s')
ax.set_ylabel('1 / ([A] / mol dm$^{-3}$)')

Looking back at the integrated rate laws we discussed earlier, we can see that this line essentially represents the integrated rate law equation, which is $\frac{1}{[A]} = \frac{1}{[A]_0} + kt$. We can see that the intercept corresponds to the value of $\frac{1}{[A]_0}$, and the gradient corresponds to the value of $k$, which is our rate constant.

We can also see from the table that the units of $k$ are $M^{-1} s^{-1}$ (or $mol^{-1} dm^{3} s^{-1}$) for a second order reaction.

Let's verify this from the rate equation's units:

$rate = k[A]^2$

$mol dm^{-3} s^{-1} = (k) * (mol dm^{-3})^2$

$mol dm^{-3} s^{-1} = (k) * (mol dm^{-3}) * (mol dm^{-3})$

Cancelling out:

$s^{-1} = (k) * (mol dm^{-3})$

$(k) = s^{-1} / (mol dm^{-3})$

$(k) = (mol^{-1} dm^{3}) s^{-1}$

So the units of k are indeed $mol^{-1} dm^{3} s^{-1}$.

$\frac{1}{[A]} = \frac{1}{[A]_0} + kt$

Substituting the known constants:

$\frac{1}{[A]} = 0.201 + 0.208t$

$y=mx+c$

Therefore $rate = 0.208[A]^2$

So say we wanted to find the concentration of reactant A at 120 seconds:

$\frac{1}{[A]} = 0.201 + (0.208 * 120)$

$[A] = \frac{1}{0.201 + (0.208 * 120)}$

$\approx$ $0.0411$ $mol$ $dm^{-3}$

In [ ]:
# Run this cell as it is
pred_A = 1 / (model.coef_[0] + (model.intercept_ * 120))
print(f"The predicted concentration of reactant A at 120 seconds is {pred_A} mol dm⁻³")

**Question (2): From inspection of the initial concentration vs time graph what do you estimate the concentration of reactant A to be at 20 seconds? Type in the code (using the above to help) and see if it matches the estimation.**

*Add some text to the cell below and press the &#9205; Run button (play symbol). To run all the cells again in order press the &#9193; button (fast-forward symbol) and then select the red "Restart and Run All Cells" option.*

In [ ]:
## ADD TEXT HERE


**Question (3): What are the limitations of predicting at particularly low or high temperatures?**

*Discuss this in groups*